In [ ]:
import os
from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain

In [ ]:
#load_dotenv()
#os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
#os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
#os.environ["LANGCHAIN_TRACKING_V2"]="true"

In [ ]:
llm = ChatOllama(
    model="llama3.2:latest",
    temperture=0.7
)

## Simple Gen AI App using Langchain
This tutorial was orginially meant for OpenApi, but I converted to Ollama as we went.

### Data Ingestion - scrape Langchain
https://docs.smith.langchain.com/administration/tutorials/manage_spend

In [ ]:
loader=WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")

docs=loader.load()

### Load Data -> text -> split -> embeed -> store in Vector DB

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents=text_splitter.split_documents(docs)


In [ ]:
embeddings=OllamaEmbeddings(model="nomic-embed-text:latest")
vectorstoredb=FAISS.from_documents(docs, embeddings)

In [ ]:
#vectorstoredb.save_local("../Embeddings/simpleapp_index")

### Query from a the vector db

#### Only use similarity search not LLM

In [ ]:
query = "What is the purpose of the tutorial?"
result=vectorstoredb.similarity_search(query)
result[0].page_content

#### Use LLM

In [ ]:
prompt=ChatPromptTemplate.from_template(
   """
    Answser the following question based only on the provided context:
    <context>
    {context}
    </context>
"""
)

In [ ]:
document_chain=create_stuff_documents_chain(llm, prompt)

document_chain.invoke({
    "input":"What does this tutorial walk through?",
    "context":[Document(page_content="This tutorial walks through optimizing your spend on LangSmith. In it, we will learn how to optimize existing spend and prevent future overspend on a realistic real-world example. We will use an existing LangSmith organization with high usage. Concepts can be transferred to your own organization.")]
})


In [ ]:
retriever=vectorstoredb.as_retriever()
retrieval_chain=create_retrieval_chain(retriever, document_chain)


In [ ]:
response=retrieval_chain.invoke({"input":"What does this tutorial walk through?"})
response['answer']


In [ ]:
response['context']